In [1]:
#https://motius.breezy.hr/p/0b622aa1163e01-join-our-tech-community-as-working-student-m-f-d/apply?utm_source=pt-linkedin&utm_medium=job-posting

In [2]:
"""!pip install datasets
!pip install transformers
!pip install sentence_transformers
!pip install peft
!pip install huggingface_hub
!pip install --upgrade accelerate
!pip install --upgrade bitsandbytes
!pip install --upgrade torchmetrics
!pip install --upgrade deepspeed
"""

'!pip install datasets\n!pip install transformers\n!pip install sentence_transformers\n!pip install peft\n!pip install huggingface_hub\n!pip install --upgrade accelerate\n!pip install --upgrade bitsandbytes\n!pip install --upgrade torchmetrics\n!pip install --upgrade deepspeed\n'

In [3]:
from transformers import  AutoModel,AutoTokenizer,AutoModelForMaskedLM , Trainer,TrainingArguments,\
BitsAndBytesConfig,pipeline,default_data_collator,DataCollatorWithPadding,DataCollatorForLanguageModeling
from transformers.utils import move_cache
#from chat_template_utils import get_json_schema
#from llama_cpp import Llama
from utils import *
from peft import *
import datasets
#import torchmetrics
import torch
from torch.utils.data.dataloader import DataLoader,Dataset
#from hqq.engine.hf import HQQModelForCausalLM
#from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
#import deepspeed
import os
import pandas as pd
import json
#from var_dump import var_dump
from bs4 import BeautifulSoup,PageElement,Comment
import matplotlib.pyplot as plt
import seaborn as sns

cache_dir='/var/tmp/.cache/' #'/proj/ciptmp/ix05ogym/.cache/'
output_dir = cache_dir+'outputs/'

move_cache(cache_dir)


No ROCm runtime is found, using ROCM_HOME='/usr'
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


0it [00:00, ?it/s]

In [4]:
train_dataset_2 = pd.read_excel('query_element_5.xlsx',index_col=0)
train_dataset_2 = train_dataset_2.dropna().drop_duplicates(['query','element','label']).reset_index(drop=True)
c = train_dataset_2["label"] == "latitude"
train_dataset_2.loc[c, "label"] = 'distance'
c = train_dataset_2["label"] == "longitude"
train_dataset_2.loc[c, "label"] = 'distance'
c = train_dataset_2["label"] == "english language level"
train_dataset_2.loc[c, "label"] = 'English language level'
c = train_dataset_2["label"] == "job"
train_dataset_2.loc[c, "label"] = 'job title'
c = train_dataset_2["label"] == "title"
train_dataset_2.loc[c, "label"] = 'person title'
#difference = pd.concat([b, a]).drop_duplicates(keep=False)
#b.to_excel
#print(set(b['label'].unique())-( person.keys() ))


In [5]:
train_dataset = pd.read_excel("train_forms.xlsx")[["input_text", "labels"]]

train_dataset.drop_duplicates(["input_text"]).reset_index(drop=True)
c = train_dataset["labels"] == "other"
train_dataset.loc[c, "labels"] = pd.NA
c = train_dataset["labels"] == "eligible"
train_dataset.loc[c, "labels"] = pd.NA
c = train_dataset["labels"] == "birthdate"
train_dataset.loc[c, "labels"] = "birth"
c = train_dataset["labels"] == "resident country"
train_dataset.loc[c, "labels"] = "country"
c = train_dataset["labels"] == "residentcountry"
train_dataset.loc[c, "labels"] = "country"
c = train_dataset["labels"] == "citizen country"
train_dataset.loc[c, "labels"] = "country"
c = train_dataset["labels"] == "recommendation"
train_dataset.loc[c, "labels"] = "recommender"
c = train_dataset["labels"] == "german language level"
train_dataset.loc[c, "labels"] = "German language level"
c = train_dataset["labels"] == "english language level"
train_dataset.loc[c, "labels"] = "English language level"



train_dataset["labels"].unique()
train_dataset = train_dataset.dropna().reset_index(drop=True)

# c = train_dataset['labels']=='privacy'
# train_dataset.loc[c,:]

lll = []
person = {
    "first name": "majid",
    "last name": "sharghi foroushani",
    "pronoun": "he",
    "birth": "10/08/1997",
    "age": "27",
    "sex": "male",
    "skill": "NLP,LLM",
    "country": "Germany",
    "address": "Philip-9",
    "city": "Erlangen",
    "phone number": "+497773154",
    "available from": "6/24/2024",
    "salary": "10000",
    "German language level": "A1",
    "English language level": "C1",
    "visa": "I do not need visa.",
    "email": "sharghi.majid@gmail.com",
    "password": "123456",
    "find us": "Google",
    "xing": "www.xing.com",
    "job title": "I am a stduent.",
    "university": "FAU",
    "major": "AI",
    "privacy": "privacy",
    "search": "search",
    "submit": "submit",
    "cookie": "cookie",
    "cover letter": "cover letter",
    "linkedin": "www.linkedin.com",
    "github": "www.github.com",
    "login": "login",
    "register": "register",
    "cv": "/pah/to/cv.pdf",
    "twitter": "www.twitter.com",
    "years of experience": "4 years in deep learning",
    "username":"Majidsh97",
    "website language":"english",
    "transcript":"path/to/ts.pdf",
    "company":"company",
    "house number":"9",
    "italian language level":"A2",
    "ethnicity":"white",
    "picture":"path/to/p.jpg",
    "career level":"entry",
    "zip":"91052",
    "newsletter":"newsletter",
    "fake":"fake",
    "question":"question",
    "cancel":"cancel",
    "recommender":"my recommender",
    "website":"my website",
    'workherebefore':"No",
    "next":"next",
    "dropbox":"www.dropbox.com",
    "googledrive":"googledrive",
    "work hours":"8",
    "workduration":"6 months",
    "willing to relocate":"yes",
    "universityenrollment":"path to enrollment letter",
    "subscribe":"subscribe",
    "distance":"50 km",
    "captcha":"captcha",
    "forgot password":"no",
    "graduation year":"2024",
    "civic number":"54864",
    "person title":"Mr",
    "show password":"show password",
    
    
}
# {'years of experience', 'forgot password', 'next', 'username', 'website language', 'latitude', 'distance', 'transcript', 'english language level', 'cancel', 'company', 'longitude', 'house number', 'italian language level', 'title', 'captcha', 'civic number', 'zip', 'recommender', 'newsletter', 'show password', 'ethnicity', 'graduation year', 'job title', 'subscribe', 'picture', 'career level'}


print(train_dataset["labels"].value_counts())


labels
privacy                   63
search                    43
email                     38
cv                        37
submit                    28
first name                25
phone number              25
cover letter              24
linkedin                  23
last name                 21
find us                   15
cancel                    12
address                   11
salary                    10
available from            10
github                     9
question                   8
transcript                 8
login                      5
birth                      5
recommender                5
German language level      4
xing                       4
password                   4
country                    4
next                       4
city                       3
register                   3
company                    3
website                    3
picture                    3
zip                        3
house number               3
cookie                     3
workher

In [6]:
test_dataset = pd.read_excel('forms.xlsx')
c = test_dataset['label'].notna()
test_dataset = test_dataset.loc[c,:].reset_index(drop=True)
test_dataset['label'].unique()

array(['cv', 'first name', 'last name', 'email', 'phone number',
       'available from', 'github', 'privacy', 'submit', 'search',
       'country', 'sex', 'zip', 'cover letter', 'linkedin', 'visa',
       'xing', 'newsletter'], dtype=object)

In [7]:
from transformers import Qwen2ForCausalLM
device = 'cuda'
model_name = 'Qwen/Qwen2-0.5B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)
model = Qwen2ForCausalLM.from_pretrained(model_name #Qwen/Qwen2-0.5B-Instruct
                                         #,attn_implementation="flash_attention_2"
                                         #,torch_dtype=torch.float16
                                         , device_map='auto',cache_dir=cache_dir).to('cuda')

model.eval()
#model.config['use_sliding_window']=True
print(model.config)

model

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Qwen2Config {
  "_name_or_path": "Qwen/Qwen2-0.5B-Instruct",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "max_position_embeddings": 32768,
  "max_window_layers": 24,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_theta": 1000000.0,
  "sliding_window": 32768,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.41.2",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}



Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Linear(in_featur

In [8]:
model.generation_config

GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "repetition_penalty": 1.1,
  "temperature": 0.7,
  "top_k": 20,
  "top_p": 0.8
}

In [9]:
model = get_peft_model(model,LoraConfig(task_type=TaskType.CAUSAL_LM,target_modules=['q_proj','v_proj'
                                                                                     ,'k_proj'#,'o_proj'
                                                                                     ]))
model.print_trainable_parameters()
model
for n,p in model.named_parameters():
    if p.requires_grad==False:
        print(n)

trainable params: 737,280 || all params: 494,770,048 || trainable%: 0.1490
base_model.model.model.embed_tokens.weight
base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight
base_model.model.model.layers.0.self_attn.q_proj.base_layer.bias
base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight
base_model.model.model.layers.0.self_attn.k_proj.base_layer.bias
base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight
base_model.model.model.layers.0.self_attn.v_proj.base_layer.bias
base_model.model.model.layers.0.self_attn.o_proj.weight
base_model.model.model.layers.0.mlp.gate_proj.weight
base_model.model.model.layers.0.mlp.up_proj.weight
base_model.model.model.layers.0.mlp.down_proj.weight
base_model.model.model.layers.0.input_layernorm.weight
base_model.model.model.layers.0.post_attention_layernorm.weight
base_model.model.model.layers.1.self_attn.q_proj.base_layer.weight
base_model.model.model.layers.1.self_attn.q_proj.base_layer.bias
base_model.model.model

In [10]:
#test_dataset.to_excel('forms.xlsx',index=0)

In [11]:
x = test_dataset['query'] + test_dataset['element']
x = x.tolist()
x[0]

'Personal information\n CV or resume\n Browse CV to autocomplete your application. You can still fill your profile manually.\n <input accept="application/pdf,.pdf" name="cv" tabindex="-1" type="file"/>'

In [12]:
s='['
for i in LABEL_INDEX_TO_KEY:
    s+=f'"{i}",'
    pass

s+=']'
s
#str(LABEL_INDEX_TO_KEY[0])

'["birth","major","cookie","search","city","English language level","years of experience","forgot password","next","username","address","website language","latitude","distance","cv","phone number","transcript","english language level","register","pronoun","university","cancel","job","company","longitude","age","house number","skill","italian language level","visa","title","twitter","captcha","civic number","zip","find us","recommender","available from","linkedin","submit","email","salary","newsletter","first name","German language level","show password","last name","ethnicity","github","cover letter","graduation year","sex","password","privacy","login","job title","subscribe","picture","country","xing","career level",]'

In [13]:
"""def f(x):
    print(x)
    return '{"answer":"'+ x +'"}'

train_dataset['labels'] = train_dataset['labels'].map(f)
train_dataset"""


'def f(x):\n    print(x)\n    return \'{"answer":"\'+ x +\'"}\'\n\ntrain_dataset[\'labels\'] = train_dataset[\'labels\'].map(f)\ntrain_dataset'

In [14]:
import random


def get_random():
    d = {}
    for k in person.keys():
        d[k] = str(random.randint(1000, 10000))

    return d


# get_random()
def person_to_str(p):
    s = "{"
    for k, v in p.items():
        s += f'"{k}":"{v}",'
        pass

    s += "}"
    return s


def get_one_chat_template(person_str, query_element_str, label_str=None):
    d = [
        {   #{"role":"system","content":""}
            "role": "user",
            #"content": """Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""
            "content": """Your answer has to be in json format like {"answer":""}."""
            
            + person_str,
        },
        {
            "role": "user",
            "content": "What is this HTML element about?\n" + query_element_str,
            #"content": "Question:\n" + query_element_str,
            
        },
    ]
    if label_str:
        d.append(
            {"role": "assistant", "content": '{"answer":"' + label_str + '"}'},
        )
        
    return d


def get_one_msg(text, label):
    p = person #get_random()
    strp = ""#person_to_str(p)
    rc = ""  # random.choice(list(p.keys()))+' '
    d = get_one_chat_template(strp, rc + text,label)#p[label])
    return d


def get_chat():

    d = []
    d+=train_dataset.apply( lambda x :get_one_msg(x["input_text"], x["labels"]),axis=1).tolist()
    d+=train_dataset_2.apply( lambda x :get_one_msg(x["query"]+'\n'+x['element'], x["label"]),axis=1).tolist()


    return d  # ,labels


get_chat()

[[{'role': 'user',
   'content': 'Your answer has to be in json format like {"answer":""}.'},
  {'role': 'user',
   'content': 'What is this HTML element about?\npersonal informationcv or resumebrowse cv to autocomplete your application. you can still fill your profile manually.<input name="cv" accept="application/pdf,.pdf" type="file" tabindex="-1" style="display: none;">'},
  {'role': 'assistant', 'content': '{"answer":"cv"}'}],
 [{'role': 'user',
   'content': 'Your answer has to be in json format like {"answer":""}.'},
  {'role': 'user',
   'content': 'What is this HTML element about?\n<input data-testid="textinput" id="email" required="" type="email" autocomplete="email" name="email" class="sc-bsgiji jskkmm" value="">'},
  {'role': 'assistant', 'content': '{"answer":"email"}'}],
 [{'role': 'user',
   'content': 'Your answer has to be in json format like {"answer":""}.'},
  {'role': 'user',
   'content': 'What is this HTML element about?\n*<input data-testid="firstname" id="firstna

In [15]:
 
#BeautifulSoup(train_dataset['input_text'][0],'html.parser').find()

In [16]:
def get_train_data():
    return tokenizer.apply_chat_template(get_chat(),add_generation_prompt=False ,truncation=True,max_length=512)  # ,return_tensors='pt')
    pass

tokenized_data = get_train_data()


no_label = tokenizer.apply_chat_template([  get_one_chat_template( ""#person_to_str(person)
                                                                  , str(x['query'])+x['element']) 
                                for i,x in test_dataset.iterrows()] ,truncation=True,max_length=512)  # ,return_tensors='pt')

print(len(tokenized_data))

print(tokenizer.decode(token_ids=tokenized_data[0]))
print(len(tokenized_data[0]))
print(tokenizer.decode(token_ids=no_label[0]))


1525
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Your answer has to be in json format like {"answer":""}.<|im_end|>
<|im_start|>user
What is this HTML element about?
personal informationcv or resumebrowse cv to autocomplete your application. you can still fill your profile manually.<input name="cv" accept="application/pdf,.pdf" type="file" tabindex="-1" style="display: none;"><|im_end|>
<|im_start|>assistant
{"answer":"cv"}<|im_end|>

99
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Your answer has to be in json format like {"answer":""}.<|im_end|>
<|im_start|>user
What is this HTML element about?
Personal information
 CV or resume
 Browse CV to autocomplete your application. You can still fill your profile manually.
 <input accept="application/pdf,.pdf" name="cv" tabindex="-1" type="file"/><|im_end|>



In [17]:
len(tokenized_data[0])

99

In [18]:
tokenizer(['<|im_start|>assistant'])

{'input_ids': [[151644, 77091]], 'attention_mask': [[1, 1]]}

In [19]:
mydc = DataCollatorForLanguageModeling(tokenizer,False)
def mmf(x):
    y = mydc(x)
    b = (y['labels']==77091).int().argmax()-1
    for i in range(0,b):
        y['labels'][0][i]=-100
        pass
    #print( y['labels'] )
    return y

mmf(no_label)['labels'].shape


torch.Size([100, 512])

In [20]:
from transformers import TrainerCallback
"""
class ChangeDatasetCallback(TrainerCallback):
    def __init__(self):
        #super.__init__()
        pass

    def on_epoch_end(self, args, state, control, **kwargs):
        # Move to the next dataset at the end of each epoch
        #self.current_epoch += 1
        #new_dataset = self.datasets[self.current_epoch % len(self.datasets)]
        kwargs['train_dataloader'].dataset = get_train_data()
        #kwargs['eval_dataloader'].dataset = new_dataset['validation']

"""

class custom_dataset(Dataset):
    def __init__(self) -> None:
        super().__init__()
        self.dataset = get_train_data()
        self.len = len(self.dataset)
        pass

    def __getitem__(self, i) :
        if i==self.len-1:
            self.dataset = get_train_data()

        return self.dataset[i]
        pass

    def __len__(self):
        return self.len
        pass





trainer =  Trainer(model,
        train_dataset=custom_dataset(),
        data_collator=mmf,
        #compute_metrics=compute_metric,
        #callbacks=[ChangeDatasetCallback()],
        args=TrainingArguments(
          output_dir=output_dir,
          num_train_epochs=4,
          per_device_train_batch_size=1,
          per_device_eval_batch_size=1,
          logging_steps=300,

          #eval_accumulation_steps =2,


        ))

trainer.train()
model = model.eval()

[2024-07-04 06:22:18,645] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


Step,Training Loss
300,0.676900
600,0.368900
900,0.257300
1200,0.246100
1500,0.188600
1800,0.165400
2100,0.169300
2400,0.152100
2700,0.166800
3000,0.176000


/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: F

KeyboardInterrupt: 

In [23]:
trainer.model.save_pretrained(output_dir+'saved_84/')


/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [22]:
questions = [
"use html to create email"
,"""what is your gender?"""
,"""where are you living?(optional)"""
,"what is your salary expectation?*"
,"Which pronouns should we use to address you? (optional)"
,"how is your German?"
,"do you need visa spunsership?"
,"where are you working right now?"
,"what is your birth date?"
,"what are your hobbies?"
,"password"
,"what projects have you done?"
,"are you a text ai?"
,"what experice do you have in network engineering?"
,"what projects have you done in deep learning?"
,"how did you find us?"]

model.disable_adapter()

x= [get_one_chat_template(""#person_to_str(person)
                          ,q+"""<input type="text" style="" id="custom">""") for q in questions]
tokenized_questions = tokenizer.apply_chat_template(x)

with torch.no_grad():
    
    for i,x in enumerate(tokenized_questions):
            x = torch.tensor([x]).to('cuda')
            o = model.generate(input_ids=x,max_new_tokens=20,do_sample=False)
            print(questions[i])
            decoded = tokenizer.decode(o[0,x.shape[1]+3:])
            print(decoded)
            print('--------------------------------------------------------')
        
    

use html to create email
{"answer":"email"}<|im_end|>
--------------------------------------------------------
what is your gender?
{"answer":"find us"}<|im_end|>
--------------------------------------------------------
where are you living?(optional)
{"answer":"location"}<|im_end|>
--------------------------------------------------------
what is your salary expectation?*
{"answer":"salary"}<|im_end|>
--------------------------------------------------------
Which pronouns should we use to address you? (optional)
{"answer":"first name"}<|im_end|>
--------------------------------------------------------
how is your German?
{"answer":"German language"}<|im_end|>
--------------------------------------------------------
do you need visa spunsership?
{"answer":"visa"}<|im_end|>
--------------------------------------------------------
where are you working right now?
{"answer":"find us"}<|im_end|>
--------------------------------------------------------
what is your birth date?
{"answer":"bir

In [21]:
model = model.to('cuda')
def get_answer(decoded):
    try:
                #print("ERROR: ",decoded)

                _s = decoded.find('{')
                _e = decoded.find('}',_s)+1
                #print(decoded[_s:_e])
                answer = json.loads(decoded[_s:_e])['answer']
                return answer
                
    except Exception as e:
                print("ERROR: ",e)
                print(decoded)
                print("---------------------------------")

                
def compute_metric():
    model.eval()
    acc=0
    with torch.no_grad():
        for i in range(len(no_label)):
            x = mydc([no_label[i]])['input_ids'].to('cuda')
            #print(x)
            o = model.generate(input_ids=x,max_new_tokens=20,do_sample=False)
            decoded = tokenizer.decode(o[0,x.shape[1]:])
            #print(decoded)
            #break

            #print(x[0],x[1])
            answer = get_answer(decoded)
            _label = test_dataset.loc[i,'label']
            #_label = person[test_dataset.loc[i,'label']]
            c = answer==_label

            if c:
                acc +=1
            else:
                print("NO_CORRECT:",answer,_label)
                print("---------------------------------")

                pass

    acc/=len(no_label)
    print("acc:",acc)



compute_metric()

/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:537: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


NO_CORRECT: pronoun sex
---------------------------------
ERROR:  Expecting value: line 1 column 1 (char 0)
="country_chile"> Chile</option><option value="country_china"> China</option><
---------------------------------
NO_CORRECT: None country
---------------------------------
NO_CORRECT: cv cover letter
---------------------------------
NO_CORRECT: pronoun sex
---------------------------------
NO_CORRECT: captcha privacy
---------------------------------
NO_CORRECT: privacy sex
---------------------------------
NO_CORRECT: search submit
---------------------------------
NO_CORRECT: find us cv
---------------------------------
NO_CORRECT: linkedin xing
---------------------------------
NO_CORRECT: job title country
---------------------------------
ERROR:  Expecting value: line 1 column 1 (char 0)
><option value="34">burgenland</option><option value="35">c
---------------------------------
NO_CORRECT: None country
---------------------------------
NO_CORRECT: pronoun sex
------------

In [ ]:
model.eval()
i =11
d = mydc(no_label[i:i+1])['input_ids'].to(device)
d = tokenizer.apply_chat_template( [#{"role":"system","content":""}
                             {"role":"user", "content":"""Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""+s}
                            ,{"role":"user", "content":'email'}] , return_tensors='pt').to(device)
#d = tokenizer(['json {"answer":""} what is this html element about? <input type="email">'],return_tensors='pt')['input_ids'].to(device)
#print(d)
with torch.no_grad():

    o = model.generate(input_ids = d,max_length = 512,do_sample=True)
    print(tokenizer.decode(o[0]))



<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Your answer has to be in json format like {"answer":""}. Use this content in your answer:
["birth","major","cookie","search","city","English language level","years of experience","forgot password","next","username","address","website language","latitude","distance","cv","phone number","transcript","english language level","register","pronoun","university","cancel","job","company","longitude","age","house number","skill","italian language level","visa","title","twitter","captcha","civic number","zip","find us","recommender","available from","linkedin","submit","email","salary","newsletter","first name","German language level","show password","last name","ethnicity","github","cover letter","graduation year","sex","password","privacy","login","job title","subscribe","picture","country","xing","career level",]<|im_end|>
<|im_start|>user
email<|im_end|>
<|im_start|>assistant
{"answer":"johndoe@example.com"}<|im_end|>

In [ ]:
$$$

LABEL_INDEX_TO_KEY

SyntaxError: invalid syntax (115010059.py, line 1)

In [ ]:
from sentence_transformers  import SentenceTransformer
#from transformers import AutoModel

embdeding_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L12-v2',cache_dir=cache_dir)

#embeding_model= AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2',cache_dir=cache_dir)#.to('cuda')
embeding_model= SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2',cache_folder=cache_dir)#.to('cuda')


embeding_model.eval()
embeding_model

In [ ]:
data = pd.read_excel('query_element_3.xlsx')
data

In [ ]:
embeding_model = get_peft_model(embeding_model,LoraConfig(target_modules=['query','value']))
embeding_model.print_trainable_parameters()


In [ ]:

#c = train_dataset['labels']=='relocate'
#train_dataset.loc[c,'labels'] = 'willing to relocate'
#train_dataset[c]


In [ ]:
#train_dataset.to_excel('train_forms.xlsx',index=0)

In [ ]:
from sentence_transformers.trainer import SentenceTransformerTrainer,SentenceTransformerTrainingArguments
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [ ]:
data.loc[:,'label']=1
train_dataset = datasets.Dataset.from_pandas(data)
train_dataset

In [ ]:
def batch_tok(example):
    example['query']= embdeding_tokenizer(example['query'])
    example['element']= embdeding_tokenizer(example['element'])

train_dataset.map(batch_tok)

In [ ]:
def batch_tok(example):

    return embdeding_tokenizer(example,padding=True,truncation=True,max_length=512,return_tensors='pt')



LABEL_INDEX_TO_KEY = list(KEYWORDS.keys())

query_token = embdeding_tokenizer(data['query'].tolist(),padding=False,truncation=True,max_length=512)
element_token = embdeding_tokenizer(data['element'].tolist(),padding=False,truncation=True,max_length=512)
classes_token = embdeding_tokenizer(LABEL_INDEX_TO_KEY,padding=True,truncation=True,max_length=512,return_tensors='pt')


def myheatmap(x):
    map =[]
    for i in range(len(x)):
        sim = torch.cosine_similarity(x,x[i])
        map.append(sim)


    plt.figure(figsize=(10,10))
    sns.heatmap(map)
    plt.show()



with torch.no_grad():

    output = embeding_model(**classes_token)
    #class_embdeding = torch.mean(output.last_hidden_state,1)
    class_embdeding = mean_pooling(output,classes_token['attention_mask'])
    class_embdeding = torch.nn.functional.normalize(class_embdeding, p=2, dim=1)
    #sim = torch.cosine_similarity(class_embdeding,class_embdeding[0])

    #print(sim.shape)
    pass

i = 151
with torch.no_grad():
    x = torch.tensor(element_token['input_ids'][i:i+1])
    #print(x)
    output = embeding_model(input_ids= x)
    #query_embdeding = torch.mean(output.last_hidden_state,1)
    query_embdeding = mean_pooling(output,torch.tensor(element_token['attention_mask'][i:i+1]))
    #query_embdeding = torch.nn.functional.normalize(class_embdeding, p=2, dim=1)

    sim = torch.cosine_similarity(class_embdeding,query_embdeding)
    print(pd.DataFrame({'value':sim,'label':LABEL_INDEX_TO_KEY}).sort_values('value',ascending=False)[:5])
    print(LABEL_INDEX_TO_KEY[ sim.argmax() ], embdeding_tokenizer.decode(token_ids=x[0]))
    print(data['element'][i] )


#myheatmap(class_embdeding)


In [ ]:
embeding_model.tokenizer = embdeding_tokenizer
embeding_model.tokenize = embdeding_tokenizer.tokenize

from sentence_transformers import SentenceTransformerModelCardData
from sentence_transformers.losses import CosineSimilarityLoss
#embeding_model.model_card_data = SentenceTransformerModelCardData(train_datasets=train_dataset)

In [ ]:
trainer  =  SentenceTransformerTrainer(model=embeding_model,
                           train_dataset=train_dataset,
                           tokenizer=embdeding_tokenizer,
                           loss=CosineSimilarityLoss(embeding_model),
                           args=SentenceTransformerTrainingArguments(
                               output_dir=output_dir,
                               num_train_epochs=1,
                               per_device_train_batch_size=1
                           )

                           )


trainer.train()

In [ ]:
test_dataset

In [ ]:
def compute_metric():
    embeding_model.eval()
    acc=0

    with torch.no_grad():

        l=embeding_model.encode(LABEL_INDEX_TO_KEY)
        l = torch.tensor(l)
        print(l.shape)

        for i,x in test_dataset.iterrows():
            #te = embdeding_tokenizer([x['element']],max_length=512,truncation=True,return_tensors='pt')['input_ids']

            o= embeding_model.encode( x['element'])
            o = torch.tensor(o)
            sim = torch.cosine_similarity(l,o)
            #print(sim.shape)
            #sim = torch.mean(sim,1)
            if LABEL_INDEX_TO_KEY[sim.argmax()]==x['label']:
                acc+=1
                pass
            #print(LABEL_INDEX_TO_KEY[sim.argmax()],x['label'])




    acc/=len(test_dataset)
    print("acc:",acc)



compute_metric()

In [ ]:
$$$

In [ ]:
with open('./html/1.html','r',encoding='utf-8') as f:
    html = f.read()


html = BeautifulSoup(html,'html.parser')

In [ ]:
def preprocess_form(form):
    if isinstance(form,str):
        form = BeautifulSoup(form,'html.parser')

    form:PageElement
    comments=form.find_all(string=lambda text: isinstance(text, Comment))
    for comment in comments:
        comment.extract()
    all_tags = form.find_all()

    # Remove all tags except for 'link', 'input', and 'select' tags
    i=0
    for element in all_tags:
        element:PageElement
        for k in ['style','class','bounding_box_rect','is_clickable']:
            if k in element.attrs:
                    del element[k]
                    pass

        if 'type' in element.attrs and element.attrs['type']=='hidden':
            element.unwrap()
            pass


        #if element.name  in ['div','text']:
        if element.name not in ['a', 'input', 'select','radio','button','textarea','checkbox','option']:

            element.unwrap()
            #print(element)
            pass
        else:
            #element['backend_node_id']=i
            pass



            i+=1

    #print(form.prettify() )
    return form

import re

def get_query_element(f):
    query_element=[]
    mytext  = ''
    texts = []

    for element in f:
        # Get the text before the element
        #print('-----------------------------')
        if element.name == None:
            t = element.text.replace('\n','').replace('\t','').strip()
            if t!='' and len(t)>2:
                #mytext+=t+'\n '
                texts.append(t)
        else:
            mytext = '\n'.join(texts[-1:])
            query_element.append({'query':mytext,'element': str(element).replace('\n','').replace('  ','').strip()})
            mytext =''
            texts=[]

    return query_element


def get_query_html(f):
    f:PageElement
    childs = f.find_all(string=True,recursive=False)
    for c in childs:
        print('---------------------')
        print(c)
    pass



print(forms.loc[10,'form'])
f = preprocess_form(forms.loc[25,'form'])
d = get_query_element(f)
d = pd.DataFrame(d)
for i,x in d.iterrows():
    print(x['query'])
    print(x['element'])
    print('--------------------------------------------------')

In [ ]:
forms=pd.read_excel('forms copy.xlsx',index_col=0).reset_index(drop=True)

In [ ]:
for _ ,f in forms.iterrows():
    print(_)
    #print(f['form'])
    f = str(f['form'])
    #print(f)
    #f = BeautifulSoup(str(f),'html.parser')
    f = preprocess_form(f)
    #f = f.find_all()
    #print(f)
    d = get_query_element(f)
    d = pd.DataFrame(d)
    #print(d)
    for i,x in d.iterrows():
        print(x['query'])
        print(x['element'])
        print('--------------------------------------------------')


    print('***************************************************')
    input()

In [ ]:
qe = pd.read_excel('query_element_4.xlsx')
qe

In [ ]:
model.eval()
toks = tokenizer.apply_chat_template([ [#{"role":"system","content":""}
                                {"role":"user", "content":"""Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""+to_str(person)}
                               ,{"role":"user", "content":"What is this HTML element about?\n"+str(x['query'])+x['element'] }
                               #,{"role":"assistant","content":""}
                               ] for i,x in qe.iterrows()] ,truncation=True,max_length=512)  # ,return_tensors='pt')





In [ ]:
from ast import keyword
import json
with torch.no_grad():
  for i in range(0,len(toks)):
      t = toks[i]
      t = torch.tensor([t]).cuda()

      o = model.generate(input_ids=t,max_new_tokens=20,do_sample=False,)
      decoded = tokenizer.decode(o[0][t.shape[1]:])
      #print(decoded)

      try:
                _s = decoded.find('{')
                _e = decoded.find('}',_s)+1
                #print(decoded[_s:])
                #print(decoded[_s:_e])
                answer = json.loads(decoded[_s:_e])['answer']
                print(i,answer)

                for k,v in person.items():
                    if v==answer:

                      qe.loc[i,'label'] = k
                    else:
                      qe.loc[i,'label'] = pd.NA

                qe.loc[i,'answer']=answer


                pass
      except Exception as e:
                print("ERROR: ",decoded)
                print("---------------------------------")

                print(e)

                pass


In [ ]:
def auto_label(x):
    element = BeautifulSoup(x['element'],'html.parser').find()
    #print(element)
    type = element.attrs.get('type')
    print(type)
    if type=='email':
        return 'email'
    elif type=='search':
        return 'search'
    elif type=='password':
        return 'password'
    elif type=='tel':
        return 'phone number'
    return pd.NA
    
    
value_to_key = {}
for k,v in person.items():
    value_to_key[v]=k
    
"""
for i,x in rows.iterrows():
    #l  = auto_label(x)
    try:
        #l = value_to_key[x['answer']]
        #qe.loc[i,'label'] = l
        #print(l)
        pass
    except:
        #qe.loc[i,'label'] = pd.NA
        pass
    #print(label)
    
"""
    

In [ ]:
person
rows

In [ ]:
rows = rows.drop_duplicates(['query','label'])
import time
for i,x in rows.iterrows():
    print(i)
    print(x['query'])
    print(x['element'])
    time.sleep(1)
    qe.loc[i,'label']=input()


In [ ]:
qe = pd.read_excel('query_element_4.xlsx')

In [ ]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 5)

#qe['label'].value_counts()
c1 =qe['label'].notna()
#c2 = qe['auto_label']=='password' 
rows = qe[c1].drop_duplicates(['query'])

rows
#qe[qe['auto_label']=='email']

#qe[qe['auto_label'].notna()]

In [ ]:
qe.loc[348,'label']='picture'
qe.loc[349,'label']='recommender'
qe.loc[350,'label']='major'

qe.loc[4,'label']='subscribe'
qe.loc[15,'label']='search'
qe.loc[44,'label']='address'
qe.loc[58,'label']='cover letter'
qe.loc[366,'label']='cover letter'
qe.loc[410,'label']='cover letter'
qe.loc[412,'label']='cover letter'




qe.loc[65,'label']=pd.NA
qe.loc[309,'label']='cover letter'
qe.loc[319,'label']='show password'
qe.loc[749:753,'label']='transcript'
qe.loc[789,'label']='university'
qe.loc[790,'label']='university'

qe.loc[1587,'label']='ethnicity'

In [ ]:
qe.loc[426,'label']='career level'
qe.loc[433,'label']='career level'
qe.loc[1305,'label']='career level'
qe.loc[1481,'label']='career level'

qe.loc[154,'label']='newsletter'
qe.loc[1480,'label']='newsletter'
qe.loc[1196,'label']='newsletter'
qe.loc[1530,'label']='sex'



qe.loc[1286,'label']='register'
qe.loc[320,'label']='login'

qe.loc[150,'label']='company'
qe.loc[1171,'label']='company'

qe.loc[320,'label']='login'
qe.loc[60,'label']='captcha'
qe.loc[63,'label']='captcha'
qe.loc[698,'label']='captcha'
qe.loc[734,'label']='captcha'



qe.loc[320,'label']='login'


qe.loc[1586,'label']='ethnicity'
qe.loc[81,'label']='skill'
qe.loc[1586,'label']='ethnicity'
qe.loc[1504,'label']='find us'
qe.loc[689,'label']='find us'
qe.loc[1532,'label']='sex'
qe.loc[1548,'label']='sex'

qe.loc[1291:1298,'label']='distance'
qe.loc[93,'label']='submit'
qe.loc[189,'label']='cancel'


qe.loc[500,'label']= pd.NA
qe.loc[501,'label']= pd.NA
qe.loc[146,'label']= pd.NA
qe.loc[295,'label']= pd.NA











In [ ]:
from sentence_transformers.similarity_functions import cos_sim 
from IPython.display import clear_output

with torch.no_grad():
    for i,x in rows.iterrows():
        _q = embeding_model.encode(x['query'].lower())
        _l = embeding_model.encode(x['label'])
        sim = cos_sim(_q,_l)
        c = sim<0.4
        if c==True:
            print(i)
            print(sim)
            print(x['query'])
            print(x['element'])
            print(x['label'])
            print('--------------------------------------------------')
            #input()
            #clear_output(True)

        #break


In [ ]:
qe.loc[32,'label']='cookie'
qe.loc[267,'label']='longitude'
qe.loc[1038,'label']='privacy'
qe.loc[1008,'label']='forgot password'
qe.loc[1452,'label']='forgot password'
qe.loc[1437,'label']='forgot password'
qe.loc[1321,'label']='forgot password'
qe.loc[98,'label']='forgot password'
qe.loc[1319,'label']='username'


qe.loc[929,'label']='recommender'
qe.loc[464,'label']='captcha'
qe.loc[369,'label']='captcha'

qe.loc[873,'label']='username'
qe.loc[250,'label']='last name'
qe.loc[155,'label']='pronoun'
qe.loc[141,'label']='pronoun'

qe.loc[123,'label']='job title'
qe.loc[763,'label']='job title'
qe.loc[273,'label']='job title'
qe.loc[229,'label']='job title'


qe.loc[102,'label']='sex'
qe.loc[101,'label']='sex'

qe.loc[125,'label']='available from'
qe.loc[1329,'label']='available from'

qe.loc[135,'label']='graduation year'
qe.loc[812,'label']='graduation year'

qe.loc[533,'label']='years of experience'
qe.loc[1274,'label']='salary'
qe.loc[1325,'label']='salary'
qe.loc[1330,'label']='salary'
qe.loc[551,'label']='salary'

qe.loc[1178,'label']='cover letter'
qe.loc[1132,'label']='cover letter'
qe.loc[1057,'label']='cover letter'
qe.loc[1032,'label']='cover letter'
qe.loc[1030,'label']='cover letter'
qe.loc[1032,'label']='cover letter'
qe.loc[1032,'label']='cover letter'






qe.loc[1494,'label']='major'
qe.loc[1490,'label']='major'

qe.loc[810,'label']='major'

qe.loc[1577,'label']='zip'
qe.loc[889,'label']='zip'

qe.loc[544,'label']='house number'
qe.loc[426,'label']='career level'
qe.loc[101,'label']='sex'
qe.loc[1277,'label']='picture'
qe.loc[1124,'label']='cancel'
qe.loc[101,'label']='sex'











In [ ]:
x = qe.drop_duplicates(['query','label']).reset_index(drop=True)
print(len(x))

print("ssss")

In [ ]:
#qe.to_excel('query_element_4.xlsx',index=0)

In [ ]:
qe.loc[146,'element']

In [ ]:
model.save_pretrained(output_dir+'saved/')



In [ ]:
x =Qwen2ForCausalLM.from_pretrained('Qwen/Qwen2-0.5B-Instruct',cache_dir=cache_dir)
model = PeftModelForCausalLM.from_pretrained(x,output_dir+'saved/')